## This notebook contains very basic SQL commands on a toy dataset
- Creating Table
- Inserting data into the table
- Query database with SELECT, JOIN, AGGREGATE functions, numerical logic and grouping commands

## Import and installation section

In [ ]:
!pip install sqllite3

In [2]:
from sqlalchemy import create_engine
import pandas as pd
import sqlite3

In [3]:
# initialize connection to the database
conn = sqlite3.connect('dog_cat.db')

# create cursor 
cur = conn.cursor()

## 1. Let's play with Dogs (& SQL)

In [4]:
# query to create table
query = '''CREATE TABLE parents (parent VARCHAR(20), child VARCHAR(20));'''

# execute the query
cur.execute(query)

In [5]:
# insert data into the table
cur.execute('''INSERT INTO parents (parent, child)
                            VALUES ("abraham", "barack") UNION
                            VALUES ("abraham", "clinton") UNION
                            VALUES ("delano", "herbert") UNION
                            VALUES ("fillmore", "abraham") UNION
                            VALUES ("fillmore", "delano") UNION
                            VALUES ("fillmore", "grover") UNION
                            VALUES ("eisenhower", "fillmore");''')

#### 1.a SELECT all records in the table

In [6]:
pd.read_sql_query('SELECT * FROM parents;', conn).head()

,parent,child
0,abraham,barack
1,abraham,clinton
2,delano,herbert
3,eisenhower,fillmore
4,fillmore,abraham


##### 1.b SELECT child and parent, where abraham is the parent

In [7]:
query = 'SELECT * FROM parents WHERE parent = "abraham";'

pd.read_sql_query(query, conn)

,parent,child
0,abraham,barack
1,abraham,clinton


##### 1.c SELECT all children that have an 'e' in their name (hint: use LIKE and '%e%')

In [8]:
query = 'SELECT child FROM parents WHERE child LIKE "%e%";'

pd.read_sql_query(query, conn)

,child
0,herbert
1,fillmore
2,delano
3,grover


##### 1.d SELECT all unique parents (use SELECT DISTINCT) and order them by name, descending order (i.e. fillmore first)

In [9]:
query = 'SELECT DISTINCT parent FROM parents ORDER BY parent DESC;'

pd.read_sql_query(query, conn)

,parent
0,fillmore
1,eisenhower
2,delano
3,abraham


##### 1.e SELECT all dogs that are siblings (one-to-one relations). Only show a sibling pair once. To do this you need to select two times from the parents table.

In [10]:
query = 'SELECT a.child AS sibling1, b.child AS sibling2 FROM parents AS a, parents AS b WHERE a.parent=b.parent AND a.child<b.child'

pd.read_sql_query(query, conn)

,sibling1,sibling2
0,barack,clinton
1,abraham,delano
2,abraham,grover
3,delano,grover


## 2 Joins

In [12]:
cur.execute('''CREATE TABLE dogs AS
                  SELECT "abraham" AS name, "long" AS fur UNION
                  SELECT "barack", "short" UNION
                  SELECT "clinton", "long" UNION
                  SELECT "delano", "long" UNION
                  SELECT "eisenhower", "short" UNION
                  SELECT "fillmore", "curly" UNION
                  SELECT "grover", "short" UNION
                  SELECT "herbert", "curly";''')

##### 2.a COUNT the number of long haired dogs

In [13]:
query = '''SELECT COUNT(fur) FROM dogs WHERE fur = "long";'''

pd.read_sql_query(query, conn)

,COUNT(fur)
0,3


##### 2.b JOIN tables parents and dogs and SELECT the parents of curly dogs

In [14]:
query = '''SELECT parents.parent FROM parents
          left JOIN dogs ON dogs.name = parents.child WHERE dogs.fur = "curly";'''

pd.read_sql_query(query, conn)

,parent
0,eisenhower
1,delano


##### 2.c JOIN tables parents and dogs, and SELECT the parents and children that have the same fur type. Only show them once

In [15]:
query = """SELECT A.name as parent, B.name as child, A.fur as fur FROM parents as P
           JOIN dogs A on P.parent = A.name
           JOIN dogs B on P.child = B.name
           WHERE A.fur = B.fur
           """
pd.read_sql_query(query, conn)

,parent,child,fur
0,abraham,clinton,long


## Q3 Aggregate functions, numerical logic and grouping

In [16]:
query = '''CREATE TABLE animals as
           SELECT "dog" as kind, 4 as legs, 20 as weight UNION
           SELECT "cat" , 4 , 10 UNION
           SELECT "ferret" , 4 , 10 UNION
           SELECT "parrot" , 2 , 6 UNION
           SELECT "penguin" , 2 , 10 UNION
           SELECT "t-rex" , 2 , 12000;'''

cur.execute(query)

##### 3.a SELECT the animal with the minimum weight. Display kind and min_weight.

In [17]:
query = 'SELECT kind, weight FROM animals ORDER BY weight ASC LIMIT 1;'

pd.read_sql_query(query, conn)

,kind,weight
0,parrot,6


##### 3.b Use the aggregate function AVG to display a table with the average number of legs and the average weight

In [18]:
query = 'SELECT AVG(legs), AVG(weight) FROM animals ORDER BY weight ASC LIMIT 1;'

pd.read_sql_query(query, conn)

,AVG(legs),AVG(weight)
0,3.0,2009.333333


##### 3.c SELECT the animal kind(s) that have more than two legs, but weighs less than 20. Display kind, weight, legs.

In [20]:
query = 'SELECT kind FROM animals WHERE legs > 2 AND weight < 20;'

pd.read_sql_query(query, conn)

,kind
0,cat
1,ferret


##### 3.d SELECT the average weight for all the animals with 2 legs and the animals with 4 legs (by using GROUP BY).

In [21]:
query = 'SELECT AVG(weight), legs FROM animals GROUP BY legs ORDER BY legs;'

pd.read_sql_query(query, conn)

,AVG(weight),legs
0,4005.333333,2
1,13.333333,4
